# Demo: Serenity Portfolio Analytics API

Basic demonstration of how to run portfolio performance statistics and Brinson attribution on a rich portfolio.

## Setup

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
from os import getenv
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(getenv('SERENITY_CONFIG_ID', None))

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# set Seaborn style
sns.set(style="whitegrid")

# create an alias to the api
api = connect_widget.get_api()

In [ ]:
from serenity_types.valuation.core import (
    AssetPosition,
    AssetWeight,
    BrinsonAttributionRequest,
    PortfolioAnalyticRequest,
    PortfolioFromAllocationRequest,
    PortfolioFromTradesRequest,
    PortfolioTimeseriesAndTrades,
    PositionTimeseries,
    RebalancingFrequency,
    Trades,
    Transfers
)

asset_master = api.refdata().load_asset_master()
btc_asset_id = asset_master.get_asset_id_by_symbol('BTC')
eth_asset_id = asset_master.get_asset_id_by_symbol('ETH')

## Creating portfolios

The new portfolio analytics API supports a much richer representation of portfolios, and has helper mechanisms that let you create portfolios from different types of inputs:

- initial cash position plus asset allocation rules
- initial positions, trades and transfers

### From allocations

In the first case you "trade" an initial cash position by periodically rebalancing the portfolio so the percentage weights are preserved. In the second case you start with a set of asset positions, possibly empty, and trade and transfer assets in and out, updating positions over time. In both cases the output will be timeseries of positions and trades.

In [ ]:
request = PortfolioFromAllocationRequest(
    initial_weights=[ AssetWeight(asset_id=btc_asset_id, weight=0.5),
                      AssetWeight(asset_id=eth_asset_id, weight=0.5) ],
    initial_cash_quantity=1_000_000,
    start_datetime=datetime(2022, 1, 1, 0, 0, 0),
    end_datetime=datetime(2022, 12, 31, 0, 0, 0),
    rebalancing_frequency=RebalancingFrequency.DAILY
)
resp_obj = api.portfolio_analytics().create_portfolio_from_allocation(request)
result = resp_obj.result
asset1 = asset_master.get_symbol_by_id(result.positions.asset_id[0], symbology='SERENITY')
asset2 = asset_master.get_symbol_by_id(result.positions.asset_id[1], symbology='SERENITY')
df = pd.DataFrame(result.positions.quantity, columns=[asset1, asset2])
df['As Of Time'] = pd.to_datetime(result.positions.as_of_time)
df.set_index('As Of Time', inplace=True)

# Create a line chart using Seaborn
plt.figure(figsize=(10, 6))
sns.lineplot(data=df, markers=True)

# Set labels and title
plt.title('Index Composition')

# Show the plot
plt.show()


### From trades

This variation creates a portfolio from several different combinations of inputs:

- positions, no trades or transfers: this takes current positions back through time, held constant
- positions, with trades and transfers: this builds up a portoflio over time from transactions

In [ ]:
trades = Trades(
    trade_datetime=[],
    base_asset_id=[],
    quote_asset_id=[],
    quantity=[],
    fill_price=[],
    commission_in_usd=[]
)
transfers = Transfers(
    transfer_datetime=[],
    asset_id=[],
    quantity=[]
)
request = PortfolioFromTradesRequest(
    initial_positions=[ AssetPosition(asset_id=btc_asset_id, quantity=1),
                        AssetPosition(asset_id=eth_asset_id, quantity=10) ],
    start_datetime=datetime(2022, 1, 1, 0, 0, 0),
    end_datetime=datetime(2022, 12, 31, 0, 0, 0),
    trades=trades,
    transfers=transfers
)
resp_obj = api.portfolio_analytics().create_portfolio_from_trades(request)
result = resp_obj.result

## Computing performance statistics

In [ ]:
try:
    positions = PositionTimeseries(as_of_time=[], asset_id=[], quantity=[])
    portfolio = PortfolioTimeseriesAndTrades(positions=positions)
    request = PortfolioAnalyticRequest(portfolio=portfolio)

    api.portfolio_analytics().compute_portfolio_statistics(request)
except:
    print("Not yet ready")


## Computing performance attribution (Brinson)

In [ ]:
try:
    positions = PositionTimeseries(as_of_time=[], asset_id=[], quantity=[])
    portfolio = PortfolioTimeseriesAndTrades(positions=positions)
    request = BrinsonAttributionRequest(portfolio=portfolio)

    api.portfolio_analytics().compute_portfolio_performance_attribution(request)
except:
    print("Not yet ready")